In [2]:
import pandas as pd
import numpy as np
import re
from urllib.parse import urlparse
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from imblearn.over_sampling import SMOTE
import joblib
import pickle
import os

# Load data
df = pd.read_csv('training_data.csv')
print(f"✅ Loaded {len(df)} rows")

# ===== FEATURE ENGINEERING =====
features = pd.DataFrame()

# Basic payload features
features['payload_length'] = df['payload'].str.len()
features['payload_word_count'] = df['payload'].str.split().str.len()
features['payload_has_url'] = df['payload'].str.contains(r'http[s]?://', regex=True, na=False).astype(int)
features['payload_has_short_url'] = df['payload'].str.contains(r'bit\.ly|tinyurl|goo\.gl|t\.co', regex=True, na=False).astype(int)
features['payload_has_ip'] = df['payload'].str.contains(r'\b\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}\b', regex=True, na=False).astype(int)

# Phishing keywords
phishing_keywords = ['urgent', 'verify', 'suspended', 'click', 'immediately', 'limited time', 
                    'winner', 'prize', 'claim', 'update', 'secure', 'alert', 'action required']
for keyword in phishing_keywords:
    features[f'has_{keyword.replace(" ", "_")}'] = df['payload'].str.contains(keyword, case=False, na=False).astype(int)

# URL features
url_pattern = r'http[s]?://([^\s]+)'
urls = df['payload'].str.extract(url_pattern, expand=False)
features['url_length'] = urls.str.len().fillna(0)
features['url_has_hyphen'] = urls.str.contains('-', na=False).astype(int)
features['url_has_digits'] = urls.str.contains(r'\d', regex=True, na=False).astype(int)

# Port features
features['port'] = pd.to_numeric(df['port'], errors='coerce').fillna(0)
features['is_common_port'] = features['port'].isin([80, 443, 25, 587, 993, 22]).astype(int)
features['is_suspicious_port'] = features['port'].isin([8080, 8000, 6667, 21, 23]).astype(int)

# IP features
ip_parts = df['source_ip'].str.split('.', expand=True).astype(float)
features['ip_first_octet'] = ip_parts[0].fillna(0)
features['is_private_ip'] = ((ip_parts[0] == 192) | (ip_parts[0] == 10) | ((ip_parts[0] == 172) & (ip_parts[1] >= 16) & (ip_parts[1] <= 31))).astype(int)

# Destination features
features['destination_is_email'] = df['destination'].str.contains('@', na=False).astype(int)

# Text patterns - FIXED: escape special regex characters
features['has_exclamation'] = df['payload'].str.count(r'!')
features['has_question_mark'] = df['payload'].str.count(r'\?')  # FIXED: escaped ?
features['uppercase_ratio'] = df['payload'].str.findall(r'[A-Z]').str.len() / (features['payload_length'] + 1)
features['number_ratio'] = df['payload'].str.findall(r'\d').str.len() / (features['payload_length'] + 1)

# Payload type encoding
payload_type_encoder = LabelEncoder()
features['payload_type_encoded'] = payload_type_encoder.fit_transform(df['payload_type'])

# Fill NaN
features = features.fillna(0)

print(f"✅ Created {features.shape[1]} features")
print(f"Feature columns: {list(features.columns)[:10]}...")

# ===== PREPARE TARGET =====
y = df['label'].map({'malicious': 1, 'benign': 0})
print(f"✅ Target: {y.value_counts().to_dict()}")

# ===== HANDLE CLASS IMBALANCE =====
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(features, y)
print(f"✅ After SMOTE: {len(X_resampled)} samples")

# ===== SPLIT DATA =====
X_train, X_test, y_train, y_test = train_test_split(
    X_resampled, y_resampled, test_size=0.2, random_state=42, stratify=y_resampled
)
print(f"✅ Train: {len(X_train)}, Test: {len(X_test)}")

# ===== TRAIN MODEL =====
model = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
model.fit(X_train, y_train)
print("✅ Model trained")

# ===== EVALUATE =====
y_pred = model.predict(X_test)
print("\n📊 Model Performance:")
print(f"Accuracy:  {accuracy_score(y_test, y_pred):.4f}")
print(f"Precision: {precision_score(y_test, y_pred):.4f}")
print(f"Recall:    {recall_score(y_test, y_pred):.4f}")
print(f"F1 Score:  {f1_score(y_test, y_pred):.4f}")
print("\n" + classification_report(y_test, y_pred))

# ===== SAVE MODEL =====
os.makedirs('../backend/ml_model', exist_ok=True)

joblib.dump(model, '../backend/ml_model/model.pkl')
with open('../backend/ml_model/feature_names.pkl', 'wb') as f:
    pickle.dump(list(features.columns), f)
with open('../backend/ml_model/payload_type_encoder.pkl', 'wb') as f:
    pickle.dump(payload_type_encoder, f)

print("\n✅ Model saved to ../backend/ml_model/model.pkl")

# ===== FEATURE IMPORTANCE =====
feature_importance = pd.DataFrame({
    'feature': features.columns,
    'importance': model.feature_importances_
}).sort_values('importance', ascending=False)

print("\n🔝 Top 10 Most Important Features:")
print(feature_importance.head(10))

✅ Loaded 145 rows
✅ Created 32 features
Feature columns: ['payload_length', 'payload_word_count', 'payload_has_url', 'payload_has_short_url', 'payload_has_ip', 'has_urgent', 'has_verify', 'has_suspended', 'has_click', 'has_immediately']...
✅ Target: {0: 84, 1: 61}
✅ After SMOTE: 168 samples
✅ Train: 134, Test: 34
✅ Model trained

📊 Model Performance:
Accuracy:  0.9412
Precision: 0.9412
Recall:    0.9412
F1 Score:  0.9412

              precision    recall  f1-score   support

           0       0.94      0.94      0.94        17
           1       0.94      0.94      0.94        17

    accuracy                           0.94        34
   macro avg       0.94      0.94      0.94        34
weighted avg       0.94      0.94      0.94        34


✅ Model saved to ../backend/ml_model/model.pkl

🔝 Top 10 Most Important Features:
                 feature  importance
25         is_private_ip    0.359100
24        ip_first_octet    0.136048
0         payload_length    0.092367
18            ur